# Chapter 8: Triangles and Centrality

Triangle counting using A² ⊙ A (element-wise multiply of A-squared with A).

In [ ]:
import graphblas as gb
from graphblas import Matrix, Vector, semiring, binary
import networkx as nx
import matplotlib.pyplot as plt

## Undirected Graph with Triangles

In [ ]:
# Create undirected graph with known triangles
edges = [(0,1), (1,2), (0,2),   # Triangle 0-1-2
         (2,3), (3,4), (2,4)]   # Triangle 2-3-4

rows, cols = zip(*edges)
# Make symmetric (undirected)
all_rows = list(rows) + list(cols)
all_cols = list(cols) + list(rows)

A = Matrix.from_coo(all_rows, all_cols, [1]*len(all_rows), 
                    nrows=5, ncols=5, dtype=int)
print("Symmetric adjacency matrix:")
print(A)

In [ ]:
G = nx.Graph(edges)
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, font_size=16)
plt.title("Graph with 2 triangles")
plt.show()

## Triangle Counting: A² ⊙ A

In [ ]:
# A² counts 2-hop paths
A2 = A.mxm(A, semiring.plus_times).new()
print("A² (2-hop path counts):")
print(A2)

In [ ]:
# A² ⊙ A: where 2-hop paths AND direct edges exist = triangles
# Each triangle is counted 6 times (once per directed edge, both directions)
triangle_matrix = A2.ewise_mult(A, binary.times).new()
print("A² ⊙ A (triangle indicators):")
print(triangle_matrix)

In [ ]:
# Total triangles = sum(A² ⊙ A) / 6
total = triangle_matrix.reduce_scalar(binary.plus).get()
num_triangles = total // 6
print(f"Total triangles: {num_triangles}")

## Per-Node Triangle Count

In [ ]:
# Sum each row: triangles involving each node
# Each triangle counts twice per node (once for each neighbor in the triangle)
node_triangles = triangle_matrix.reduce_rowwise(binary.plus).new()
print("Raw triangle counts per node:")
print(node_triangles)

In [ ]:
# Divide by 2 for actual count per node
node_triangles_actual = node_triangles.apply(binary.truediv, right=2).new()
print("Triangles per node:")
indices, values = node_triangles_actual.to_coo()
for i, v in zip(indices, values):
    print(f"  Node {i}: {int(v)} triangles")

print("\nNode 2 has 2 triangles (0-1-2 and 2-3-4)")

## Triangle Centrality

In [ ]:
# Nodes in more triangles are more central in clustered communities
fig, ax = plt.subplots(figsize=(8, 6))

# Color nodes by triangle count
indices, values = node_triangles_actual.to_coo()
tri_counts = {int(i): int(v) for i, v in zip(indices, values)}
colors = [tri_counts.get(i, 0) for i in range(5)]

nx.draw(G, pos, with_labels=True, node_color=colors, cmap=plt.cm.YlOrRd,
        node_size=700, font_size=16, ax=ax)
ax.set_title("Triangle Centrality (darker = more triangles)")

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=plt.cm.YlOrRd, 
                           norm=plt.Normalize(vmin=min(colors), vmax=max(colors)))
sm.set_array([])
plt.colorbar(sm, ax=ax, label='Triangles')
plt.show()